In [27]:
from parameters import FileParams as fp
import os
import numpy as np
import neuralmodels

In [28]:
def get_param_data(params: str):
    print('Loading data for params: ', params)
    train_x = np.load(os.path.join(fp.data_dir, f'np_data_{params}_params', 'train_x.npy'))
    train_y = np.load(os.path.join(fp.data_dir, f'np_data_{params}_params', 'train_y.npy'))
    print(f'Percent of data positive: {np.mean(train_y) * 100:.2f}%')
    print('train_x size: ', train_x.nbytes / 1024 / 1024, 'MB')
    print(len(train_x))
    print()

    val_x = np.load(os.path.join(fp.data_dir, f'np_data_{params}_params', 'val_x.npy'))
    val_y = np.load(os.path.join(fp.data_dir, f'np_data_{params}_params', 'val_y.npy'))
    print(f'Percent of data positive: {np.mean(val_y) * 100:.2f}%')
    print('val_x size: ', val_x.nbytes / 1024 / 1024, 'MB')
    return train_x, train_y, val_x, val_y

In [54]:
from keras.callbacks import TensorBoard, ModelCheckpoint


BATCH_SIZE = 128
EPOCHS = 4


def create_and_train_model(model_name: str, model_funct, funct_args, data, batch_size=BATCH_SIZE, epochs=EPOCHS):
    model_checkpoint_callback = ModelCheckpoint(
        filepath=f'./checkpoints/{model_name}',
        save_weights_only=False,  # Save the whole model
        monitor='val_accuracy',
        mode='auto',
        save_best_only=True)

    log_dir = os.path.join(fp.log_dir, model_name)

    if funct_args is not None:
        my_model = model_funct(*funct_args)
    else:
        my_model = model_funct()
    my_model.summary()

    train_x, train_y, val_x, val_y = data
    
    my_model.fit(
        train_x,
        train_y,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(val_x, val_y),
        callbacks=[TensorBoard(log_dir), model_checkpoint_callback]
    )

In [47]:
mycroft_data = get_param_data('mycroft')

Loading data for params:  mycroft
Percent of data positive: 19.02%
train_x size:  1975.988296508789 MB
686994

Percent of data positive: 0.03%
val_x size:  102.68038177490234 MB


In [5]:
custom1_data = get_param_data('custom1')

Loading data for params:  custom1
Percent of data positive: 23.66%
train_x size:  977.397705078125 MB
552196

Percent of data positive: 0.05%
val_y size:  33.2232666015625 MB


In [29]:
custom2_data = get_param_data('custom2')

Loading data for params:  custom2
Percent of data positive: 23.68%
train_x size:  1587.3566131591797 MB
551878

Percent of data positive: 0.05%
val_x size:  53.98780822753906 MB


In [33]:
create_and_train_model('mycroft cnn', neuralmodels.cnn_model, None, mycroft_data)

here
here
Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_7 (Normalizat  (None, 29, 13)           27        
 ion)                                                            
                                                                 
 conv1d_14 (Conv1D)          (None, 27, 32)            1280      
                                                                 
 conv1d_15 (Conv1D)          (None, 25, 64)            6208      
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 12, 64)           0         
 1D)                                                             
                                                                 
 dropout_14 (Dropout)        (None, 12, 64)            0         
                                                                 
 flatten_7 (Flatten)         (None, 768)       

INFO:tensorflow:Assets written to: ./checkpoints\mycroft cnn\assets


INFO:tensorflow:Assets written to: ./checkpoints\mycroft cnn\assets


5368/5368 [==============================] - 14s 3ms/step - loss: 0.0080 - accuracy: 0.9971 - val_loss: 3.6579e-04 - val_accuracy: 0.9999
Epoch 2/4
5368/5368 [==============================] - 14s 3ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 6.3861e-04 - val_accuracy: 1.0000
Epoch 3/4
5368/5368 [==============================] - 14s 3ms/step - loss: 0.0010 - accuracy: 0.9997 - val_loss: 5.9664e-04 - val_accuracy: 1.0000
Epoch 4/4
5368/5368 [==============================] - 14s 3ms/step - loss: 7.6171e-04 - accuracy: 0.9998 - val_loss: 9.4387e-04 - val_accuracy: 1.0000


In [9]:
import parameters
ap = parameters.parameters['custom1']

create_and_train_model('custom1 cnn', neuralmodels.cnn_model, (ap,), custom1_data)

Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 29, 8)            17        
 n)                                                              
                                                                 
 conv1d (Conv1D)             (None, 27, 32)            800       
                                                                 
 conv1d_1 (Conv1D)           (None, 25, 64)            6208      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 64)            0         
                                                                 
 flatten (Flatten)           (None, 768)               0 

INFO:tensorflow:Assets written to: ./checkpoints\custom1 cnn\assets


INFO:tensorflow:Assets written to: ./checkpoints\custom1 cnn\assets


4315/4315 [==============================] - 16s 3ms/step - loss: 0.0108 - accuracy: 0.9963 - val_loss: 0.0012 - val_accuracy: 0.9998
Epoch 2/4
4299/4315 [============================>.] - ETA: 0s - loss: 0.0020 - accuracy: 0.9994

INFO:tensorflow:Assets written to: ./checkpoints\custom1 cnn\assets


INFO:tensorflow:Assets written to: ./checkpoints\custom1 cnn\assets


4315/4315 [==============================] - 11s 3ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 7.9679e-04 - val_accuracy: 0.9997
Epoch 3/4
4315/4315 [==============================] - 11s 2ms/step - loss: 0.0013 - accuracy: 0.9996 - val_loss: 0.0034 - val_accuracy: 0.9995
Epoch 4/4
4299/4315 [============================>.] - ETA: 0s - loss: 0.0012 - accuracy: 0.9996

INFO:tensorflow:Assets written to: ./checkpoints\custom1 cnn\assets


INFO:tensorflow:Assets written to: ./checkpoints\custom1 cnn\assets


4315/4315 [==============================] - 11s 3ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 5.4580e-04 - val_accuracy: 0.9999


In [58]:
from parameters import AudioParams

def smaller_cnn(ap: AudioParams):
    import tensorflow as tf
    from tensorflow.keras import layers
    from tensorflow.keras import models
    # Instantiate the `tf.keras.layers.Normalization` layer.
    norm_layer = layers.Normalization()
    # Fit the state of the layer to the spectrograms
    # with `Normalization.adapt`.
    # norm_layer.adapt(data=train_spectrogram_ds.map(map_func=lambda spec, label: spec))

    input_shape = (ap.n_features, ap.n_mfcc)

    model = models.Sequential([
        layers.Input(shape=input_shape),
        # Downsample the input.
        # layers.Resizing(32, 32),
        # Normalize.
        norm_layer,
        layers.Conv1D(16, 3, activation='relu'),
        layers.Conv1D(32, 3, activation='relu'),
        layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='smaller_cnn')

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )

    return model

def larger_cnn(ap: AudioParams):
    import tensorflow as tf
    from tensorflow.keras import layers
    from tensorflow.keras import models
    # Instantiate the `tf.keras.layers.Normalization` layer.
    norm_layer = layers.Normalization()
    # Fit the state of the layer to the spectrograms
    # with `Normalization.adapt`.
    # norm_layer.adapt(data=train_spectrogram_ds.map(map_func=lambda spec, label: spec))

    input_shape = (ap.n_features, ap.n_mfcc)

    model = models.Sequential([
        layers.Input(shape=input_shape),
        # Downsample the input.
        # layers.Resizing(32, 32),
        # Normalize.
        norm_layer,
        layers.Conv1D(32, 3, activation='relu'),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='larger_cnn')

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )

    return model

import parameters
ap = parameters.mycroftParams

create_and_train_model('smaller_cnn', smaller_cnn, (ap,), mycroft_data)

Model: "smaller_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_13 (Normaliza  (None, 29, 13)           27        
 tion)                                                           
                                                                 
 conv1d_41 (Conv1D)          (None, 27, 16)            640       
                                                                 
 conv1d_42 (Conv1D)          (None, 25, 32)            1568      
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 12, 32)           0         
 g1D)                                                            
                                                                 
 dropout_52 (Dropout)        (None, 12, 32)            0         
                                                                 
 flatten_24 (Flatten)        (None, 384)               

INFO:tensorflow:Assets written to: ./checkpoints\smaller_cnn\assets


INFO:tensorflow:Assets written to: ./checkpoints\smaller_cnn\assets


5368/5368 [==============================] - 10s 2ms/step - loss: 0.0140 - accuracy: 0.9950 - val_loss: 1.3705e-04 - val_accuracy: 1.0000
Epoch 2/4
5368/5368 [==============================] - 10s 2ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 1.0538e-04 - val_accuracy: 0.9999
Epoch 3/4
5341/5368 [============================>.] - ETA: 0s - loss: 0.0016 - accuracy: 0.9995

INFO:tensorflow:Assets written to: ./checkpoints\smaller_cnn\assets


INFO:tensorflow:Assets written to: ./checkpoints\smaller_cnn\assets


5368/5368 [==============================] - 10s 2ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 4.4472e-06 - val_accuracy: 1.0000
Epoch 4/4
5368/5368 [==============================] - 10s 2ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 4.3457e-05 - val_accuracy: 1.0000


In [9]:
import parameters
ap = parameters.parameters['custom2']

create_and_train_model('custom2 cnn2', neuralmodels.cnn_model, (ap,), custom2_data)

Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 29, 13)           27        
 n)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 27, 32)            1280      
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 64)            6208      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 dropout_2 (Dropout)         (None, 12, 64)            0         
                                                                 
 flatten (Flatten)           (None, 768)               0 

INFO:tensorflow:Assets written to: ./checkpoints\custom2 cnn2\assets


INFO:tensorflow:Assets written to: ./checkpoints\custom2 cnn2\assets


4312/4312 [==============================] - 17s 3ms/step - loss: 0.0092 - accuracy: 0.9967 - val_loss: 0.0022 - val_accuracy: 0.9997
Epoch 2/4
4298/4312 [============================>.] - ETA: 0s - loss: 0.0016 - accuracy: 0.9995

INFO:tensorflow:Assets written to: ./checkpoints\custom2 cnn2\assets


INFO:tensorflow:Assets written to: ./checkpoints\custom2 cnn2\assets


4312/4312 [==============================] - 11s 3ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.0011 - val_accuracy: 0.9998
Epoch 3/4
4312/4312 [==============================] - 11s 3ms/step - loss: 9.9854e-04 - accuracy: 0.9997 - val_loss: 0.0049 - val_accuracy: 0.9995
Epoch 4/4
4312/4312 [==============================] - 11s 3ms/step - loss: 8.0031e-04 - accuracy: 0.9998 - val_loss: 0.0031 - val_accuracy: 0.9998


In [31]:
train_x, train_y, val_x, val_y = custom2_data
print(train_x.shape)
print(train_y.shape)
print(train_y[:10])

(551878, 29, 13)
(551878,)
[0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]


In [32]:
import keras
model = keras.models.load_model(r'.\checkpoints\no_norm_one_conv_no_pool_one_dropout_smaller_dense')

In [33]:
model.evaluate(val_x, val_y)

587/587 [==============================] - 1s 1ms/step - loss: 7.7879e-04 - accuracy: 0.9998


[0.0007787923677824438, 0.9998401999473572]

In [46]:
predictions = model.predict(train_x[:10])
print(predictions)
predictions = np.round(predictions)
print(predictions.flatten())
print(train_y[:10])

1/1 [==============================] - 0s 12ms/step
[[2.0205894e-13]
 [9.9999487e-01]
 [9.9999917e-01]
 [9.9999058e-01]
 [1.0000000e+00]
 [1.0000000e+00]
 [9.9999142e-01]
 [6.2534396e-05]
 [5.7105815e-08]
 [2.4434177e-16]]
[0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
[0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]


In [40]:
print(val_y[:10])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [51]:
import parameters
from parameters import AudioParams
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models


def full_model(ap: AudioParams):
    norm_layer = layers.Normalization()
    input_shape = (ap.n_features, ap.n_mfcc)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        # Normalize.
        norm_layer,
        layers.Conv1D(32, 3, activation='relu'),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='full_model')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )
    return model


def no_normalization(ap: AudioParams):
    input_shape = (ap.n_features, ap.n_mfcc)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(32, 3, activation='relu'),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='no_normalization')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )
    return model


def no_norm_one_conv(ap: AudioParams):
    input_shape = (ap.n_features, ap.n_mfcc)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(32, 3, activation='relu'),
        # layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='no_norm_one_conv')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )
    return model


def no_norm_one_conv_no_pool(ap: AudioParams):
    input_shape = (ap.n_features, ap.n_mfcc)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(32, 3, activation='relu'),
        # layers.Conv1D(64, 3, activation='relu'),
        # layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='no_norm_one_conv_no_pool')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )
    return model


def no_norm_one_conv_no_pool_one_dropout(ap: AudioParams):
    input_shape = (ap.n_features, ap.n_mfcc)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(32, 3, activation='relu'),
        # layers.Conv1D(64, 3, activation='relu'),
        # layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        # layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='no_norm_one_conv_no_pool_one_dropout')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )
    return model


def no_norm_one_conv_no_pool_one_dropout_smaller_conv(ap: AudioParams):
    input_shape = (ap.n_features, ap.n_mfcc)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(16, 3, activation='relu'),
        # layers.Conv1D(64, 3, activation='relu'),
        # layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        # layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='no_norm_one_conv_no_pool_one_dropout_smaller_conv')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )
    return model


def no_norm_one_conv_no_pool_one_dropout_smaller_dense(ap: AudioParams):
    """
    After 10 epochs, this model had the lowest training loss
    """
    input_shape = (ap.n_features, ap.n_mfcc)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(32, 3, activation='relu'),
        # layers.Conv1D(64, 3, activation='relu'),
        # layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        # layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='no_norm_one_conv_no_pool_one_dropout_smaller_dense')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )
    return model


def no_norm_one_conv_no_pool_one_dropout_smaller_dense_and_conv(ap: AudioParams):
    input_shape = (ap.n_features, ap.n_mfcc)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(16, 3, activation='relu'),
        # layers.Conv1D(64, 3, activation='relu'),
        # layers.MaxPooling1D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        # layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ], name='no_norm_one_conv_no_pool_one_dropout_smaller_dense_and_conv')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'],
    )
    return model

models_to_test = {
    # "full_model": full_model,
    # "no_normalization": no_normalization,
    "no_norm_one_conv": no_norm_one_conv,
    # "no_norm_one_conv_no_pool": no_norm_one_conv_no_pool,
    # "no_norm_one_conv_no_pool_one_dropout": no_norm_one_conv_no_pool_one_dropout,
    # "no_norm_one_conv_no_pool_one_dropout_smaller_conv": no_norm_one_conv_no_pool_one_dropout_smaller_conv,
    # "no_norm_one_conv_no_pool_one_dropout_smaller_dense": no_norm_one_conv_no_pool_one_dropout_smaller_dense,
    # "no_norm_one_conv_no_pool_one_dropout_smaller_dense_and_conv": no_norm_one_conv_no_pool_one_dropout_smaller_dense_and_conv,
}

ap = parameters.mycroftParams

for model_name, model_funct in models_to_test.items():
    create_and_train_model(model_name, model_funct, (ap,), custom2_data, epochs=10)

Model: "no_norm_one_conv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_32 (Conv1D)          (None, 27, 32)            1280      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 13, 32)           0         
 g1D)                                                            
                                                                 
 dropout_40 (Dropout)        (None, 13, 32)            0         
                                                                 
 flatten_18 (Flatten)        (None, 416)               0         
                                                                 
 dense_46 (Dense)            (None, 128)               53376     
                                                                 
 dropout_41 (Dropout)        (None, 128)               0         
                                                  

INFO:tensorflow:Assets written to: ./checkpoints\no_norm_one_conv\assets


INFO:tensorflow:Assets written to: ./checkpoints\no_norm_one_conv\assets


4312/4312 [==============================] - 7s 2ms/step - loss: 0.0157 - accuracy: 0.9945 - val_loss: 0.0025 - val_accuracy: 0.9996
Epoch 2/10
4305/4312 [============================>.] - ETA: 0s - loss: 0.0026 - accuracy: 0.9992

INFO:tensorflow:Assets written to: ./checkpoints\no_norm_one_conv\assets


...

INFO:tensorflow:Assets written to: ./checkpoints\no_norm_one_conv\assets


4312/4312 [==============================] - 7s 2ms/step - loss: 0.0013 - accuracy: 0.9997 - val_loss: 0.0025 - val_accuracy: 0.9998
Epoch 10/10
4308/4312 [============================>.] - ETA: 0s - loss: 0.0010 - accuracy: 0.9997

INFO:tensorflow:Assets written to: ./checkpoints\no_norm_one_conv\assets


INFO:tensorflow:Assets written to: ./checkpoints\no_norm_one_conv\assets


4312/4312 [==============================] - 7s 2ms/step - loss: 0.0010 - accuracy: 0.9997 - val_loss: 0.0031 - val_accuracy: 0.9998


In [4]:
"""
Test different sizes of GRU units
"""
from create_models import variable_GRU_units_model


units = [4, 8, 10, 16, 20, 24]
for unit in units:
    print(f'\n\nGRU units: {unit}')
    create_and_train_model(f'model_GRU_{unit}', variable_GRU_units_model, (unit,))

"""
After testing, seems like GRU units of around 16 or 20 is best
"""



GRU units: 4
Model: "model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4)                 228       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 233
Trainable params: 233
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
5368/5368 [==============================] - 164s 30ms/step - loss: 0.1426 - accuracy: 0.9405 - val_loss: 0.0316 - val_accuracy: 0.9924
Epoch 2/4
5368/5368 [==============================] - 159s 30ms/step - loss: 0.0383 - accuracy: 0.9865 - val_loss: 0.0168 - val_accuracy: 0.9954
Epoch 3/4
5368/5368 [==============================] - 159s 30ms/step - loss: 0.0303 - accuracy: 0.9893 - val_loss: 0.0113 - val_accuracy: 0.9964
Epoch

In [10]:
"""
Test different intermediate layer sizes
"""
import create_models
import importlib
importlib.reload(create_models)

layers = [5, 8, 10]
for layer in layers:
    print(f'\n\nIntermediate layers: {layer}')
    create_and_train_model(f'model_GRU_16_layers_{layer}', create_models.variable_intermediate_layer_model, (layer,), epochs=6)



Intermediate layers: 5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_8 (GRU)                 (None, 16)                1488      
                                                                 
 dense_10 (Dense)            (None, 5)                 85        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 1,579
Trainable params: 1,579
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
5368/5368 [==============================] - 167s 31ms/step - loss: 0.0368 - accuracy: 0.9850 - val_loss: 0.0012 - val_accuracy: 0.9997
Epoch 2/6
5368/5368 [==============================] - 177s 33ms/step - loss: 0.0060 - accuracy: 0.9980 - val_loss: 0.0081 - val_accuracy: 0.9981
Epoch 3/6
5368/5368 [=====

In [15]:
"""
Load and continue training the 20 GRU model
"""

# Load model from log/model_GRU_20
from keras.models import load_model

model_GRU_20 = load_model('./checkpoints/model_GRU_20') 
model_GRU_20._name = 'model_GRU_20'  # Rename model
model_GRU_20.summary()

Model: "model_GRU_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 20)                2100      
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Continue training
def continue_training(nn_model, epochs=4, batch_size=BATCH_SIZE):
    model_name = str(nn_model.name)
    model_checkpoint_callback = ModelCheckpoint(
        filepath=f'./checkpoints/{model_name}',
        save_weights_only=False,  # Save the whole model
        monitor='val_loss',
        mode='min',
        save_best_only=True)

    log_dir = os.path.join(fp.log_dir, model_name)

    nn_model.summary()
    nn_model.fit(
        train_x,
        train_y,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(val_x, val_y),
        callbacks=[TensorBoard(log_dir), model_checkpoint_callback]
    )

continue_training(model_GRU_20, epochs=6)

Model: "model_GRU_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 20)                2100      
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
5368/5368 [==============================] - 167s 31ms/step - loss: 0.0034 - accuracy: 0.9989 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 2/6
5368/5368 [==============================] - 167s 31ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 5.5850e-04 - val_accuracy: 0.9998
Epoch 3/6
5368/5368 [==============================] - 166s 31ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 3.6128e-04 - val_accuracy: 0.9999
Ep